In [2]:
%load_ext autoreload
%autoreload 2
from imports import *
%run simcodes/MultiBandSim.py

In [62]:
stars = pd.read_csv('data/query_1610017978.44685i.txt',skiprows=6,sep='\t')
stars['field']=stars['# ID'].str.split('-').apply(lambda x:x[1])
stars['Iband']=stars['# ID'].apply(lambda x:'data/ogle/I/'+x+'.dat')
stars['Vband']=stars['# ID'].apply(lambda x:'data/ogle/V/'+x+'.dat')
stars = stars[stars['field']=='BLG']
Analysis = pd.DataFrame()
Nreps = 10
Percents = np.arange(0.5,1.,0.1)
for i in stars.index[:2]:
    print(stars.loc[i,'# ID'])
    inds = []
    try:
        Iband = pd.read_csv(stars.loc[i,'Iband'],names=['t','mag','magerr'],sep=' ')
        Iband['filt'] = 'I'
        inds.append(0)
    except:
        print('No I')
        Iband = None
    try:
        Vband = pd.read_csv(stars.loc[i,'Vband'],names=['t','mag','magerr'],sep=' ')
        Vband['filt']= 'V'
        inds.append(1)
    except:
        print('No V')
        Vband=None
    if len(inds)==2:
        D=pd.concat([Iband,Vband])
    elif len(inds)==1:
        D = [Iband,Vband][inds[0]]
    else:
        continue
    file = 'data/ogle/sources/'+stars.loc[i,'# ID']+'.dat'
    D.to_csv(file)
    N_I = (D['filt']=='I').count()
    N_V = (D['filt']=='V').count()
    Nmax = max(N_I, N_V)
    Sim = MCSimulation('data/ogle/sources/', stars.loc[i,'P_1'], file)
    sims=np.unique(np.round(Nmax*Percents).astype(int))
    Sim.produce_bootstrap(sims,Nreps)
    Sim.run_simulation('fast', )
    Stat = Sim.Simulated_periods.groupby('N').agg(['mean','std'])
    Dict = {'Name':stars.loc[i,'# ID'],'OGLE period':stars.loc[i,'P_1'],'LS period':Sim.best_fitting[1],'N_I':N_I,'N_V':N_V}
    for ind, name,Per in zip(Stat.index,sims,Percents):
        Dict[f'{name} mean'] = Stat.loc[ind,('P','mean')]
        Dict[f'{name} std'] = Stat.loc[ind,('P','std')]
    Analysis = Analysis.append(Dict,ignore_index=True)

OGLE-BLG-RRLYR-00001
OGLE-BLG-RRLYR-00002


In [59]:
Sim.Simulated_periods.groupby('N').agg(['mean','std']).loc[60,('P','std')]

0.06289062472265827

In [29]:
for i in range(5):
    if True:
        continue
    print('A')

In [58]:
{}.append({'a':5})

AttributeError: 'dict' object has no attribute 'append'

In [61]:
Analysis

,0.5 mean,0.5 std,0.6 mean,0.6 std,0.7 mean,0.7 std,0.7999999999999999 mean,0.7999999999999999 std,0.8999999999999999 mean,0.8999999999999999 std,LS period,N_I,N_V,Name,OGLE period
0,0.672040,0.191350,0.606801,0.265112,0.609385,0.259660,0.609560,0.259297,0.732540,0.000031,0.732554,106.0,106.0,OGLE-BLG-RRLYR-00001,0.732588
1,0.215021,0.092099,0.266956,0.073321,0.281551,0.063178,0.281419,0.063597,0.265983,0.075067,0.301531,119.0,119.0,OGLE-BLG-RRLYR-00002,0.301534
